# Kanawha / FFRD Metadata Demo

In [363]:
import rdflib

In [364]:
g = rdflib.Graph()
g.parse("./kanawha.ttl", format="turtle")

<Graph identifier=Na4723759b2c3410db782ca32424c0f03 (<class 'rdflib.graph.Graph'>)>

## Query for models based a model creator name
Find models created by a particlar person.

In [365]:
model_creator = "Mark McBroom"

In [366]:
query_creator = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT DISTINCT ?title ?description ?model
WHERE {{
    ?model a rascat:RasModel .
    ?model dcterms:title ?title .
    ?model dcterms:description ?description .
    ?model dcterms:creator ?creators .
    ?model dcterms:creator [foaf:name "{model_creator}"] .
}}
"""

In [367]:
results = g.query(query_creator)
for row in results:
    title = row[0]
    description = row[1]
    model = row[2]
    print(title, model)

GauleySummersville_BLE_FEMA http://example.ffrd.fema.gov/kanawha/models/GSummersville_B.prj
GauleySummersville_BLE-C_FEMA http://example.ffrd.fema.gov/kanawha/models/GSummersville_C.prj
GauleyLower_BLE-C_FEMA http://example.ffrd.fema.gov/kanawha/models/GauleyLower_BLE_FEM.prj


## Query for models based on the creator organization

Find models created by people who belong to a certain organization.

In [368]:
org_name = "Freese and Nichols"

In [390]:
query_org = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT DISTINCT ?title ?description ?model ?orgName ?jvName
WHERE {{
    ?model a rascat:RasModel .
    ?model dcterms:title ?title .
    ?model dcterms:description ?description .
    ?model dcterms:creator ?creator .
    ?creator foaf:member ?org .
    ?org foaf:name ?orgName .
    ?org foaf:member ?jv .
    ?jv foaf:name ?jvName .
    FILTER (?orgName = "{org_name}")
}}
"""

In [391]:
results = g.query(query_org)
for row in results:
    title = row[0]
    description = row[1]
    model = row[2]
    org_name = row[3]
    jv_name = row[4]
    print(title, model, org_name, jv_name)

Upper Kanawha http://example.ffrd.fema.gov/kanawha/models/UpperKanawha.prj Freese and Nichols ARC JV


## Query for models based on 2D mesh cell count
Find models with more than X cells.

In [371]:
cell_count = 400000

In [372]:
query_cell_count = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
SELECT ?title ?description ?model ?geometry ?cellCount
WHERE {{
    ?model a rascat:RasModel .
    ?model dcterms:title ?title .
    ?model dcterms:description ?description .
    ?model rascat:hasGeometry ?geometry .
    ?geometry rascat:hasMesh2D ?mesh2D .
    ?mesh2D rascat:cellCount ?cellCount .
    FILTER (?cellCount > {cell_count})
}}
ORDER BY DESC(?cellCount)    
"""

In [373]:
results = g.query(query_cell_count)
for row in results:
    title = row[0]
    description = row[1]
    model = row[2]
    geometry = row[3]
    cell_count = row[4]
    print(title, cell_count, geometry)

GauleyLower_BLE-C_FEMA 1088388 http://example.ffrd.fema.gov/kanawha/models/GauleyLower_BLE_FEM.g01
UpperNew_Upper 1064723 http://example.ffrd.fema.gov/kanawha/models/UpperNew_Upper.g01
GauleySummersville_BLE-C_FEMA 954995 http://example.ffrd.fema.gov/kanawha/models/GSummersville_C.g01
UpperNew_Lower 800701 http://example.ffrd.fema.gov/kanawha/models/UpperNew_Lower.g03
ElkMiddle 761035 http://example.ffrd.fema.gov/kanawha/models/ElkMiddle.g01
ElkMiddle 761035 http://example.ffrd.fema.gov/kanawha/models/ElkMiddle.g02
GauleySummersville_BLE_FEMA 587360 http://example.ffrd.fema.gov/kanawha/models/GSummersville_B.g01
Upper Kanawha 537254 http://example.ffrd.fema.gov/kanawha/models/UpperKanawha.g01
New-Little River 517580 http://example.ffrd.fema.gov/kanawha/models/New-LittleRiver.g01
Bluestone Local - Compass 2D BLE 417852 http://example.ffrd.fema.gov/kanawha/models/BluestoneLocal.g01
Watershed G3 411065 http://example.ffrd.fema.gov/kanawha/models/WatershedG3.g01
Watershed G3 411065 http://

## Query based on calibration metrics
Identify the top 10 Plan calibration hydrographs by according to the Nash-Sutcliffe Efficiency metric.

In [374]:
limit = 10

In [375]:
query_calibration = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX usgs_gages: <https://waterdata.usgs.gov/monitoring-location/>
SELECT ?title ?description ?model ?flow ?gage ?nse ?flowTitle ?gageTitle ?hydroType ?plan ?planTitle
WHERE {{
    ?model a rascat:RasModel .
    ?model dcterms:title ?title .
    ?model dcterms:description ?description .
    ?model rascat:hasPlan ?plan .
    ?plan rascat:hasUnsteadyFlow ?flow .
    ?plan rascat:hasCalibration ?calib .
    ?plan dcterms:title ?planTitle .
    ?flow dcterms:title ?flowTitle .
    ?calib rascat:fromStreamgage ?gage .
    ?calib rascat:hydrographType ?hydroType .
    ?gage dcterms:identifier ?gageID .
    ?gage dcterms:title ?gageTitle .
    ?hydro rascat:nse ?nse .
    FILTER (?hydroType = "Flow")
}}
ORDER BY ASC(?nse)
LIMIT {limit}
"""

In [376]:
results = g.query(query_calibration)
for row in results:
    title = row[0]
    description = row[1]
    model = row[2]
    flow = row[3]
    gage = row[4]
    nse = row[5]
    flow_title = row[6]
    gage_title = row[7]
    hydro_type = row[8]
    plan = row[9]
    plan_title = row[10]
    print(title, nse, plan_title, plan, gage)

CoalRiver -180.7 Nov2003 http://example.ffrd.fema.gov/kanawha/models/CoalRiver.p01 https://waterdata.usgs.gov/monitoring-location/03198350
CoalRiver -180.7 Nov2003 http://example.ffrd.fema.gov/kanawha/models/CoalRiver.p01 https://waterdata.usgs.gov/monitoring-location/03198500
CoalRiver -180.7 Nov2003 http://example.ffrd.fema.gov/kanawha/models/CoalRiver.p01 https://waterdata.usgs.gov/monitoring-location/03200500
CoalRiver -180.7 Jan1995 http://example.ffrd.fema.gov/kanawha/models/CoalRiver.p02 https://waterdata.usgs.gov/monitoring-location/03198500
CoalRiver -180.7 Jan1995 http://example.ffrd.fema.gov/kanawha/models/CoalRiver.p02 https://waterdata.usgs.gov/monitoring-location/03200500
CoalRiver -180.7 Jan1996 http://example.ffrd.fema.gov/kanawha/models/CoalRiver.p03 https://waterdata.usgs.gov/monitoring-location/03198500
CoalRiver -180.7 Jan1996 http://example.ffrd.fema.gov/kanawha/models/CoalRiver.p03 https://waterdata.usgs.gov/monitoring-location/03200500
CoalRiver -180.7 Jun2016 ht

## List all surface roughness landuse/landcover sources

In [377]:
query_lulc = """
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX usgs_gages: <https://waterdata.usgs.gov/monitoring-location/>
SELECT ?landuseDesc (GROUP_CONCAT(DISTINCT ?title; separator=", ") as ?titles)
WHERE {
    ?model a rascat:RasModel .
    ?model dcterms:title ?title .
    ?model dcterms:description ?description .
    ?model rascat:hasGeometry ?geom .
    ?geom rascat:hasRoughness ?rough .
    ?rough rascat:hasLanduseLandcover ?landuse .
    ?landuse dcterms:description ?landuseDesc .
}
GROUP BY ?landuseDesc
"""

In [378]:
results = g.query(query_lulc)
for row in results:
    lulc_desc = row[0]
    titles = row[1]
    print(lulc_desc)

Custom machine learning land cover analysis of NAIP 2022 imagery
National Land Cover Database 2019 (CONUS)
National Agriculture Imagery Program (NAIP) imagery processed using machine learning tools. Pulled August 2022


## Identify any models that used a certain USGS gage for calibration

In [379]:
gage = "03187500"

In [380]:
query_gage = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX usgs_gages: <https://waterdata.usgs.gov/monitoring-location/>
SELECT DISTINCT ?model ?gage ?gageID
WHERE {{
    ?model a rascat:RasModel .
    ?model rascat:hasPlan ?plan .
    ?plan rascat:hasCalibration ?calib .
    ?calib rascat:fromStreamgage ?gage .
    ?gage dcterms:identifier ?gageID .
    FILTER (?gageID = "{gage}")
}}
"""

In [381]:
results = g.query(query_gage)
for row in results:
    model = row[0]
    print(model)

http://example.ffrd.fema.gov/kanawha/models/GSummersville_B.prj
http://example.ffrd.fema.gov/kanawha/models/GSummersville_C.prj


## Map USGS gages associated with a model

In [382]:
model = "New-LittleRiver.prj"

In [383]:
query_gages = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX usgs_gages: <https://waterdata.usgs.gov/monitoring-location/>
PREFIX kanawha_models: <http://example.ffrd.fema.gov/kanawha/models/>
SELECT DISTINCT ?model ?gage ?gageID
WHERE {{
    ?model a rascat:RasModel .
    ?model rascat:hasPlan ?plan .
    ?plan rascat:hasCalibration ?calib .
    ?calib rascat:fromStreamgage ?gage .
    ?gage dcterms:identifier ?gageID .
    FILTER (?model = kanawha_models:{model})
}}
"""

In [384]:
results = g.query(query_gages)
gage_urls = []
for row in results:
    model = row[0]
    gage = row[1]
    gage_urls.append(gage)
    gageID = row[2]
    print(model, gage, gageID)

http://example.ffrd.fema.gov/kanawha/models/New-LittleRiver.prj https://waterdata.usgs.gov/monitoring-location/03170000 03170000
http://example.ffrd.fema.gov/kanawha/models/New-LittleRiver.prj https://waterdata.usgs.gov/monitoring-location/03171000 03171000


In [385]:
gage_urls

[rdflib.term.URIRef('https://waterdata.usgs.gov/monitoring-location/03170000'),
 rdflib.term.URIRef('https://waterdata.usgs.gov/monitoring-location/03171000')]

In [386]:
import json

import requests
from bs4 import BeautifulSoup

def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    soup = BeautifulSoup(req.text, parser)
    return json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))

In [387]:
gage_ld_jsons = []
for gage_url in gage_urls:
    gage_ld_jsons.append(get_ld_json(gage_url))

In [388]:
print(gage_ld_jsons)

[{'@context': ['https://opengeospatial.github.io/ELFIE/json-ld/hyf.jsonld'], '@id': 'https://waterdata.usgs.gov/monitoring-location/03170000/', '@type': 'http://www.opengeospatial.org/standards/waterml2/hy_features/HY_HydroLocation', 'name': 'LITTLE RIVER AT GRAYSONTOWN, VA', 'description': 'Monitoring location 03170000 is associated with a Stream in Pulaski County, Virginia. Current conditions of Discharge, Gage height, and Precipitation are available. Water data back to 1928 are available online.', 'sameAs': 'https://waterdata.usgs.gov/nwis/inventory/?site_no=03170000', 'HY_HydroLocationType': 'hydrometricStation', 'geo': {'@type': 'schema:GeoCoordinates', 'latitude': '37.03762635', 'longitude': '-80.5567239'}, 'image': 'https://labs.waterdata.usgs.gov/api/graph-images/monitoring-location/03170000/?parameterCode=00065'}, {'@context': ['https://opengeospatial.github.io/ELFIE/json-ld/hyf.jsonld'], '@id': 'https://waterdata.usgs.gov/monitoring-location/03171000/', '@type': 'http://www.o

In [389]:
from ipyleaflet import Map, Marker
from ipywidgets import HTML
import statistics

center_lat = statistics.mean([float(i['geo']['latitude']) for i in gage_ld_jsons])
center_lng = statistics.mean([float(i['geo']['longitude']) for i in gage_ld_jsons])

center = (center_lat, center_lng)

m = Map(center=center, zoom=9)

for j in gage_ld_jsons:
    location = (float(j['geo']['latitude']), float(j['geo']['longitude']))
    name = j['name']
    description = j['description']
    gage_url = j['@id']
    gage_popup = HTML()
    gage_popup.value = f"<h3>{name}</h3><p>Used to calibrate <b>{model}</b></p><p>{description}</p><p><a target=\"_blank\" rel=\"noopener noreferrer\" href=\"{gage_url}\">{gage_url}</a></p>"
    marker = Marker(location=location, draggable=False, title=name)
    marker.popup = gage_popup
    m.add_layer(marker)
m

Map(center=[37.089709445, -80.562974445], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…